## Яндекс Практикум, курс "Инженер Машинного Обучения" (2024 г.)
## Проект 4-го спринта: "Создание рекомендательной системы"

## Этап 4. Сервис рекомендаций

__Постановка задачи__

Напишите FastAPI-микросервис для выдачи рекомендаций, который:
- принимает запрос с идентификатором пользователя и выдаёт рекомендации,
- учитывает историю пользователя,
- смешивает онлайн- и оффлайн-рекомендации.

В README опишите стратегию смешивания онлайн- и оффлайн-рекомендаций.

Выполните тестирование микросервиса в ноутбуке `part_3_test.ipynb`:
- для пользователя без персональных рекомендаций,
- для пользователя с персональными рекомендациями, но без онлайн-истории,
- для пользователя с персональными рекомендациями и онлайн-историей.

### Инициализация

In [1]:
import numpy as np
import pandas as pd
import requests

In [2]:
import sys

# Проверяем, в каком окружении работаем
print(sys.executable)

/home/mle-user/.venv/bin/python


In [3]:
# Адреса сервисов
recommendations_url = "http://127.0.0.1:8000"
features_store_url = "http://127.0.0.1:8010"
events_store_url = "http://127.0.0.1:8020"

# Общий заголовок для всех запросов
headers = {"Content-type": "application/json", "Accept": "text/plain"}

Для запуска сервисов с помощью uvicorn необходимо выполнить 3 команды в 3-х разных терминалах, находясь в папке проекта:

```
uvicorn recommendations_service:app
uvicorn events_service:app --port 8020
uvicorn features_service:app --port 8010
```

### Загрузка данных

Загрузким треки для отображения рекомендаций со всеми полями, а также сами рекомендации, чтобы выбирать из них пользователей и объекты для тестирования

In [92]:
items = pd.read_parquet("items.parquet")
recommendations = pd.read_parquet('recommendations.parquet')
similar = pd.read_parquet('similar.parquet')

,score,item_id_1,item_id_2
1,0.924673,549,26771
2,0.923671,549,4198
3,0.921660,549,69658
4,0.921044,549,37402640
5,0.913545,549,1706463
...,...,...,...
109995,0.915183,46063416,35539575
109996,0.914844,46063416,21835466
109997,0.914437,46063416,44069660
109998,0.913806,46063416,21548470


### Получение рекомендаций по умолчанию

Напишем функцию для отображения данных

In [54]:
def display_items(item_ids):

    item_columns_to_use = ["item_id", "name", "genres", "albums", "artists"]
    
    items_selected = items.query("item_id in @item_ids")[item_columns_to_use]
    items_selected = items_selected.set_index("item_id").reindex(item_ids)
    items_selected = items_selected.reset_index()
    
    display(items_selected)

Отправляем запрос для получения рекомендаций по умолчанию, т.е. из числа топ-треков по кол-ву прослушиваний

In [84]:
params = {'k': 10}
resp = requests.post(recommendations_url + "/recommendations_default", headers=headers, params=params)

if resp.status_code == 200:
    recs = resp.json()
    item_ids = recs['recs']
    display_items(item_ids)
    
else:
    recs = []
    print(f"status code: {resp.status_code}")

,item_id,name,genres,albums,artists
0,53404,Smells Like Teen Spirit,"[alternative, allrock, rock]",[Smells Like Teen Spirit / In Bloom / On A Pla...,[Nirvana]
1,178529,Numb,"[numetal, metal]","[Meteora, 00s Rock Anthems]",[Linkin Park]
2,37384,Zombie,"[allrock, rock]","[90s Alternative, MNM Sing Your Song: Back To ...",[The Cranberries]
3,6705392,Seven Nation Army,[alternative],"[Pay Close Attention : XL Recordings, Radio 1 ...",[The White Stripes]
4,33311009,Believer,"[allrock, rock]","[Horoscope Tunes: Cancer, Workout Music Hits 2...",[Imagine Dragons]
5,328683,Bring Me To Life,"[alternative, allrock, rock]","[Mental Health 2020, Monster Millennium Hits, ...",[Evanescence]
6,148345,Californication,"[allrock, rock]","[The Studio Album Collection 1991-2011, Greate...",[Red Hot Chili Peppers]
7,795836,Shape Of My Heart,"[allrock, pop, rock]","[25 Years, Сентиментальные рок-песни, Ten Summ...",[Sting]
8,630670,"You're Gonna Go Far, Kid",[punk],"[You're Gonna Go Far, Kid, Sleep Music Rock Ed...",[The Offspring]
9,178495,In the End,"[numetal, metal]","[Antyradio: Najlepszy Rock Na Swiecie Vol. 4, ...",[Linkin Park]


### Получение персональных рекомендаций без онлайн-истории

In [130]:
# Выберем произвольного пользователя случайным образом
user_id = recommendations['user_id'].sample().iat[0]
user_id

35604

In [131]:
# Отправляем запрос для получения только оффлайн-рекомендаций
params = {'user_id': user_id, 'k': 10}
resp = requests.post(recommendations_url + "/recommendations_offline", headers=headers, params=params)
if resp.status_code == 200:
    recs = resp.json()
    display_items(recs['recs'])
else:
    recs = []
    print(f"status code: {resp.status_code}")

,item_id,name,genres,albums,artists
0,15271,Somebody Told Me,"[alternative, allrock, indie, rock]","[Essential Bands, Play Non Stop, 00's Best Of,...",[The Killers]


### Получение персональных рекомендаций с онлайн-историей

Выберем 3 произвольных трека, имеющих i2i-рекомендации, и добавим их в онлайн-историю пользователя

In [132]:
item_ids = similar['item_id_1'].unique()[:3]
print(item_ids)

for item_id in item_ids:
    params = {"user_id": user_id, "item_id": item_id}
    resp = requests.post(events_store_url + "/put", headers=headers, params=params)
    if resp.status_code == 200:
        result = resp.json()
        print(result['result'])    
    else:
        print(f"status code: {resp.status_code}") 

[549 553 558]
ok
ok
ok


Проверим, что события добавились, причем в обратном порядке

In [133]:
params = {"user_id": user_id, "k": 3}
resp = requests.post(events_store_url + "/get", headers=headers, params=params)
if resp.status_code == 200:
    result = resp.json()
    print(result['events'])    
else:
    print(f"status code: {resp.status_code}")

[558, 553, 549]


Протестируем работу сервиса `feautures_service`. 

Возмьмем последний объект из списка выше и найдем похожие на него треки из файла `similar.parquet`

In [134]:
item_id_1 = item_ids[-1]
similar.query('item_id_1 == @item_id_1')

,score,item_id_1,item_id_2
23,0.924216,558,1706463
24,0.908504,558,1706462
25,0.904736,558,550048
26,0.897181,558,69658
27,0.894441,558,553
28,0.886535,558,71239
29,0.885370,558,2277920
30,0.881694,558,71965
31,0.881250,558,34300
32,0.873259,558,10260869


Теперь получим тот же набор объектов с помощью сервиса `features_service`

In [135]:
params = {"item_id": item_id_1, "k": 10}
resp = requests.post(features_store_url +"/similar_items", headers=headers, params=params)

if resp.status_code == 200:
    result = resp.json()
    print(result)    
else:
    print(f"status code: {resp.status_code}")    

{'item_id_2': [1706463, 1706462, 550048, 69658, 553, 71239, 2277920, 71965, 34300, 10260869], 'score': [0.9242160320281982, 0.9085038900375366, 0.904736340045929, 0.89718097448349, 0.8944413065910339, 0.8865345120429993, 0.8853704929351807, 0.8816941380500793, 0.8812497854232788, 0.8732590079307556]}


Протестируем запрос к сервису `reccomendations_service` для получения объектов, похожих на 3 последних из онлайн-истории данного пользователя

In [ ]:
params = {'user_id': user_id, 'k': 10}
resp = requests.post(recommendations_url + "/recommendations_online", headers=headers, params=params)
if resp.status_code == 200:
    recs = resp.json()
    display_items(recs['recs'])
else:
    recs = []
    print(f"status code: {resp.status_code}")

Наконец, получим смешанные рекомендации. Оффлайн-рекомендации будут на нечетных местах, онлайн - на четных.

In [136]:
params = {"user_id": user_id, 'k': 10}
resp = requests.post(recommendations_url + "/recommendations", headers=headers, params=params)

if resp.status_code == 200:
    result = resp.json()
    display_items(recs['recs'])
    
else:
    result = None
    print(f"status code: {resp.status_code}")

,item_id,name,genres,albums,artists
0,15271,Somebody Told Me,"[alternative, allrock, indie, rock]","[Essential Bands, Play Non Stop, 00's Best Of,...",[The Killers]
